In [ ]:
import logging

import pandas as pd
import seaborn as sns
from scipy import stats

import divisivenormalization.utils as helpers
from divisivenormalization.data import Dataset, MonkeySubDataset

helpers.config_ipython()

logging.basicConfig(level=logging.INFO)

sns.set()
sns.set_style("ticks")
# adjust sns paper context rc parameters
font_size = 8
rc_dict = {
    "font.size": font_size,
    "axes.titlesize": font_size,
    "axes.labelsize": font_size,
    "xtick.labelsize": font_size,
    "ytick.labelsize": font_size,
    "legend.fontsize": font_size,
    "figure.figsize": (helpers.cm2inch(8), helpers.cm2inch(8)),
    "figure.dpi": 300,
    "pdf.fonttype": 42,
    "savefig.transparent": True,
    "savefig.bbox_inches": "tight",
}
sns.set_context("paper", rc=rc_dict)


class args:
    num_best = 10
    fname_best_csv = "df_best.csv"
    weights_path = "weights"
    train_logs_path = "train_logs"
    stim_full_size = 140  # full size of stimulus w/o subsampling and cropping
    stim_subsample = 2
    crop = 20



 ### Load data

In [ ]:
results_df = pd.read_csv("results.csv")
# Save a simplified version of the csv file, sorted by validation set performance
df_plain = helpers.simplify_df(results_df)
df_plain.to_csv("results_plain.csv")

data_dict = Dataset.get_clean_data()
data = MonkeySubDataset(data_dict, seed=1000, train_frac=0.8, subsample=args.stim_subsample, crop=args.crop)



 ### Get and save FEV performance on test set
 Use the 10 best models for analysis. As this operation requires model loading, we do it only if it
 was not done before.

In [ ]:
try:
    df_best = pd.read_csv(args.fname_best_csv)
    logging.info("loaded data from " + args.fname_best_csv)

except FileNotFoundError:
    df_best = df_plain[0 : args.num_best].copy()

    fev_lst = []
    for i in range(args.num_best):
        run_no = df_best.iloc[i]["run_no"]
        logging.info("load run no " + str(run_no))
        model = helpers.load_dn_model(run_no, results_df, data, args.train_logs_path)

        fev = model.evaluate_fev_testset()
        fev_lst.append(fev)

        feve = model.evaluate_fev_testset_per_neuron()
        helpers.pkl_dump(feve, run_no, "feve.pkl", args.weights_path)

        with model.session.as_default():
            u = model.u.eval()
        helpers.pkl_dump(u, run_no, "u.pkl", args.weights_path)

    df_best["fev"] = fev_lst
    df_best.to_csv(args.fname_best_csv)



In [ ]:
fev = df_best.fev.values * 100
print("Mean FEV", fev.mean())
print("SEM", stats.sem(fev, ddof=1))
print("max FEV", fev.max())
print("FEV of model with max correlation on validation set", fev[0])
